<a href="https://colab.research.google.com/github/LiamDuero03/DS-Society-Project/blob/main/2-Preliminary-EDA/Preliminary-EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LiamDuero03/DS-Society-Project/blob/main/2-Preliminary-EDA/Preliminary-EDA.ipynb)

# 2. Preliminary EDA & Visualization
Goal: Understand the "shape" and "health" of the data.

## 2.1 Understanding the "Skeleton" (Structure)
First, we use basic Pandas commands to see what we are working with. You should always start here to check for missing values or unexpected data types.

In [ ]:
# Check the shape (Rows, Columns)
print(f"Dataset Dimensions: {master_df.shape}")

# See the data types and null counts
print("\n--- Data Summary ---")
print(master_df.info())

# Get basic statistics (Mean, Min, Max) for numerical columns
print("\n--- Numerical Statistics ---")
display(master_df.describe())

NameError: name 'master_df' is not defined

## 2.2 The "Big Picture" Analysis (Pairplots)

Once we know our data is clean, we want to see how different variables interact. Instead of making dozens of individual charts, we use a **Pairplot**. This is a powerful "all-in-one" visualization that helps us spot patterns across the entire dataset instantly.

---

#### What is happening in this visualization?

A Pairplot creates a matrix of charts. To read it like a pro, look at these three distinct areas:

1. **The Diagonal (The Curves):**
Running from the top-left to the bottom-right, you see **Distribution Plots** (KDE). These show you the "shape" of a single variable—for example, where most city temperatures are concentrated.
2. **The Scatter Plots (The Dots):**
Every other square shows the relationship between **two different variables**.
* If the dots form a clear line, the variables are **correlated**.
* If the dots look like a messy cloud, there is **no relationship**.


3. **The Red Regression Lines (`kind='reg'`):**
These lines represent the "best fit" for the data.
* **Sloping Up:** As one value goes up, the other tends to go up (e.g., Humidity vs. Feels Like).
* **Sloping Down:** As one value goes up, the other tends to go down.



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 1. We select only the numerical columns that make sense for a beginner to compare
# We exclude lat/lng for now to focus on the 'Weather vs People' story
cols_to_plot = ['pop', 'temp', 'feels_like', 'humidity', 'wind']

# 2. Create the Pairplot
# 'diag_kind' shows the univariate distribution (histogram) on the diagonal
# 'kind=reg' adds a trend line to the scatter plots to show correlations
plot = sns.pairplot(
    master_df[cols_to_plot],
    diag_kind='kde',
    kind='reg',
    plot_kws={'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.4}}
)

# Add a title to the figure
plot.fig.suptitle("The 'Big Picture': Multi-Variable Relationships", y=1.02, fontsize=16)

plt.show()